In [ ]:
# Little Lemon Analysis and Sales Report

### Student Name:María José Egea Mussi
### Date: 02/08/2025

This notebook connects to the Little Lemon database to perform an analysis of bookings and sales. It demonstrates the use of connection pooling, handling multiple transactions, and using stored procedures.

**Prerequisites:** Before running this notebook, please ensure you have run the `1_setup_database.py` script to create and populate the database.

In [ ]:
# Import necessary libraries
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error, PoolError
import mysql.connector as connector

print("Libraries imported successfully.")

In [ ]:
# --- TASK 1: ESTABLISH A CONNECTION POOL ---
print("--- TASK 1: ESTABLISHING CONNECTION POOL ---")
try:
    # IMPORTANT: UPDATE WITH YOUR MYSQL CREDENTIALS
    dbconfig = {
        "database": "little_lemon_db",
        "user": "your_username",
        "password": "your_password"
    }
    
    # Create the connection pool named 'pool_b' with a size of two.
    pool_b = MySQLConnectionPool(
        pool_name="pool_b",
        pool_size=2,
        **dbconfig
    )
    print("Connection pool 'pool_b' created successfully.")
    print(f"Pool Name: {pool_b.pool_name}, Pool Size: {pool_b.pool_size}")

except Error as e:
    print(f"Error while creating connection pool: {e}")

In [ ]:
# --- TASK 2: HANDLE SIMULTANEOUS BOOKINGS ---
print("\n--- TASK 2: SIMULATING SIMULTANEOUS BOOKINGS ---")

# Guest booking information
bookings_data = [
    {'TableNo': 8, 'GuestFirstName': 'Anees', 'GuestLastName': 'Java', 'BookingSlot': '18:00:00', 'EmployeeID': 6},
    {'TableNo': 5, 'GuestFirstName': 'Bald', 'GuestLastName': 'Vin', 'BookingSlot': '19:00:00', 'EmployeeID': 6},
    {'TableNo': 12, 'GuestFirstName': 'Jay', 'GuestLastName': 'Kon', 'BookingSlot': '19:30:00', 'EmployeeID': 6}
]

insert_query = """
INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES (%(TableNo)s, %(GuestFirstName)s, %(GuestLastName)s, %(BookingSlot)s, %(EmployeeID)s);
"""

connections = []
try:
    # Get connections for the first two guests from the pool
    print("Guest 1 (Anees) trying to connect...")
    conn1 = pool_b.get_connection()
    connections.append(conn1)
    print("Guest 1 connected.")
    
    print("Guest 2 (Bald) trying to connect...")
    conn2 = pool_b.get_connection()
    connections.append(conn2)
    print("Guest 2 connected.")

    # Add a temporary connection for the third guest as the pool is full
    print("Guest 3 (Jay) trying to connect... Pool is full, adding a temporary connection.")
    pool_b.add_connection()
    conn3 = pool_b.get_connection()
    connections.append(conn3)
    print("Guest 3 connected with a new temporary connection.")

    # Create cursors and insert data for each guest
    print("\nProcessing bookings...")
    cursor1 = conn1.cursor()
    cursor1.execute(insert_query, bookings_data[0])
    conn1.commit()
    print(f"Booking for {bookings_data[0]['GuestFirstName']} is successful.")

    cursor2 = conn2.cursor()
    cursor2.execute(insert_query, bookings_data[1])
    conn2.commit()
    print(f"Booking for {bookings_data[1]['GuestFirstName']} is successful.")

    cursor3 = conn3.cursor()
    cursor3.execute(insert_query, bookings_data[2])
    conn3.commit()
    print(f"Booking for {bookings_data[2]['GuestFirstName']} is successful.")

finally:
    # Return all connections back to the pool
    print("\nReturning connections to the pool...")
    for conn in connections:
        if conn.is_connected():
            try:
                conn.close()
                print(f"Connection {conn.connection_id} returned to the pool.")
            except PoolError as e:
                # This error is expected for the third, temporary connection that was added.
                print(f"Error returning connection {conn.connection_id}: {e}. This is expected for a temporary connection.")
    print("All booking operations complete.")

In [ ]:
# --- TASK 3: GENERATE AN ANALYSIS REPORT ---
print("\n--- TASK 3: GENERATING ANALYSIS REPORT ---")
report_conn = None
try:
    report_conn = pool_b.get_connection()
    cursor = report_conn.cursor()

    # 1. Manager's name and EmployeeID
    query1 = "SELECT Name, EmployeeID FROM Employees WHERE Role = 'Manager';"
    cursor.execute(query1)
    result = cursor.fetchone()
    print("\n1. Manager Information:")
    print(f"   Name: {result[0]}, EmployeeID: {result[1]}")
        
    # 2. Employee with the highest salary
    query2 = "SELECT Name, Role FROM Employees ORDER BY CAST(Annual_Salary AS UNSIGNED) DESC LIMIT 1;"
    cursor.execute(query2)
    result = cursor.fetchone()
    print("\n2. Employee with Highest Salary:")
    print(f"   Name: {result[0]}, Role: {result[1]}")
        
    # 3. Number of guests booked between 18:00 and 20:00
    query3 = "SELECT COUNT(BookingID) FROM Bookings WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00';"
    cursor.execute(query3)
    result = cursor.fetchone()
    print(f"\n3. Number of guests booked between 18:00 and 20:00: {result[0]}")
    
    # 4. All guests waiting to be seated, sorted by booking slot
    query4 = "SELECT CONCAT(GuestFirstName, ' ', GuestLastName) AS GuestName, BookingID, BookingSlot FROM Bookings ORDER BY BookingSlot ASC;"
    cursor.execute(query4)
    results = cursor.fetchall()
    print("\n4. Guests Waiting to be Seated (Sorted by Booking Time):")
    for row in results:
        print(f"   BookingID: {row[1]}, Guest: {row[0]}, Time: {row[2]}")
        
except Error as e:
    print(f"Error generating report: {e}")
finally:
    if report_conn and report_conn.is_connected():
        cursor.close()
        report_conn.close()
        print("\nReport generation connection closed and returned to the pool.")

In [ ]:
# --- TASK 4: CREATE AND CALL 'BasicSalesReport' STORED PROCEDURE ---
print("\n--- TASK 4: BASIC SALES REPORT STORED PROCEDURE ---")
sp_conn = None
try:
    sp_conn = pool_b.get_connection()
    cursor = sp_conn.cursor()

    # SQL to create the stored procedure
    sp_drop = "DROP PROCEDURE IF EXISTS BasicSalesReport;"
    sp_create = """
    CREATE PROCEDURE BasicSalesReport()
    BEGIN
        SELECT
            SUM(BillAmount) AS TotalSales,
            AVG(BillAmount) AS AverageSale,
            MIN(BillAmount) AS MinimumBill,
            MAX(BillAmount) AS MaximumBill
        FROM Orders;
    END
    """
    
    print("Creating 'BasicSalesReport' stored procedure...")
    cursor.execute(sp_drop)
    cursor.execute(sp_create)
    print("Procedure created successfully.")
    
    # Call the procedure
    print("Calling the stored procedure...")
    cursor.callproc("BasicSalesReport")
    
    # Fetch and display results
    for result in cursor.stored_results():
        report_data = result.fetchone()
    
    print("\n--- Basic Sales Report ---")
    print(f"Total Sales: ${report_data[0]:.2f}")
    print(f"Average Sale: ${report_data[1]:.2f}")
    print(f"Minimum Bill Paid: ${report_data[2]:.2f}")
    print(f"Maximum Bill Paid: ${report_data[3]:.2f}")

except Error as e:
    print(f"Error with stored procedure: {e}")
finally:
    if sp_conn and sp_conn.is_connected():
        cursor.close()
        sp_conn.close()
        print("\nStored procedure connection closed and returned to the pool.")

In [ ]:
# --- TASK 5: DISPLAY UPCOMING BOOKINGS FOR KITCHEN SCREEN ---
print("\n--- TASK 5: KITCHEN SCREEN - UPCOMING BOOKINGS ---")
kitchen_conn = None
try:
    kitchen_conn = pool_b.get_connection()
    # A buffered cursor is good practice when you might run multiple queries.
    cursor = kitchen_conn.cursor(buffered=True)

    # Query to get the next three bookings with employee details, sorted by time
    query = """
    SELECT 
        b.BookingSlot, 
        CONCAT(b.GuestFirstName, ' ', b.GuestLastName) AS GuestName, 
        e.Name AS EmployeeName, 
        e.Role AS EmployeeRole 
    FROM Bookings b
    INNER JOIN Employees e ON b.EmployeeID = e.EmployeeID
    ORDER BY b.BookingSlot ASC
    LIMIT 3;
    """
    
    cursor.execute(query)
    upcoming_bookings = cursor.fetchall()

    print("\n--- Kitchen Display: Next 3 Bookings ---\n")
    for booking in upcoming_bookings:
        print(f"{booking[0]}")
        print(f"{booking[1]}")
        print(f"Assigned to: {booking[2]} [{booking[3]}]")
        print("-" * 25)

except Error as e:
    print(f"Error fetching upcoming bookings: {e}")
finally:
    if kitchen_conn and kitchen_conn.is_connected():
        cursor.close()
        kitchen_conn.close()
        print("Kitchen screen connection closed and returned to the pool.")